# Train example notebook

This notebook is used to implement the training of a neural network for classification of `Cloud`, `Edge`, `Good` images. <br> It is advisable to use this notebook to get practice and debug your code. To speed up the execution, once you are ready, you should move to a scripted version.

## 1. - Imports

Select `CUDA_VISIBLE_DEVICES` to the `Graphics Proceesing Unit (GPU)` index that you want to use to enable the use of GPU.

In [2]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="0" # GPU index

Enabling autoreload of different packages.

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch 
import sys
sys.path.insert(1, os.path.join("..", "data"))
sys.path.insert(1, os.path.join("..", "utils"))
from data_utils import Dataset
from plot_utils import plot_image
from torch.utils.data import DataLoader

## 2. - Datasets

### 2.1 - Creating datasets

Now we read the images from the target directory `path_data`. Set `path_data` to the directory containing the `Cloud`, `Edge`, `Good` subfolders.  Moreover, it will automatically split the total dataset into the train, cross validation and test splits by using a pseudo-random splitting algorithm. You can reproduce the split by specifying the variable `seed`. **NB**:
- The train split contains 70% of the whole images.
- The valid splits contains 15% of the whole images.
- The test splits contains 15% of the whole images.<br>**YOU MUST NOT CHANGE THE TEST SPLIT SIZE!!!**

In [5]:
# Path to the data folder (update the variable to your path).
path_data=os.path.join("..", "data")
# Seed value
seed=22

<img src="utilities/images/danger_icon.png" style="margin:auto"/>

**N.B** Make sure to have created a dataset split into the three directories `Cloud`, and `Good`, `Edge`. Otherwise, the next cell will **fail!** <br>


In [6]:
dataset=Dataset(path_data=path_data, seed=seed)
dataset.read_data()

Parsing class: Cloud: 96it [00:10,  8.75it/s]
Parsing class: Edge: 102it [00:08, 11.40it/s]
Parsing class: Good: 106it [00:10,  9.74it/s]


**Hint:** before proceeding, make sure that your `Edge`,`Cloud`, and `Good` samples are well enough among the `train`, `valid`,`test` splits. To print datasets statistics, run the next line.  Remember that the number of images in the different splits is distributed as described above. <br> If you are not happy with the data distribution, you can update the seed used and create a new dataset by rerunning the cell above. 

In [7]:
dataset.get_statistics()

,train,valid,test
cloud,68,15,13
edge,77,13,12
good,67,18,21


### 2.2. - Create data loaders.

The next lines will create a dataloader. A data loader is used to break the dataset into batches of a size `batch_size`. <br> This is useful to ensure that your dataset will fit into your memory and to create a "stochastic" implementation of gradient descent. <br> For more information, please, check: [data loader](https://www.educative.io/answers/what-is-pytorch-dataloader).<br>
Specify `batch_size` (**Hint**: use powers of 2. Typical values are between 8 and 64).

In [8]:
batch_size=16

In [9]:
# Train loader
train_loader = DataLoader(dataset.get_split("train"), batch_size=batch_size, pin_memory=False, shuffle=True)
# Cross validation data loader
valid_loader = DataLoader(dataset.get_split("valid"), batch_size=batch_size, pin_memory=False, shuffle=True)
# Test data loader
test_loader = DataLoader(dataset.get_split("test"), batch_size=batch_size, pin_memory=False, shuffle=True)

## 3 - Training

Now, it is your turn! Add your code below to load a Neural Network model, select optimizers, learning rate and perform training. <br>
Good luck!

...

In [10]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

    
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18')
    
import torch.optim as optim
optimizer = torch.optim.SGD(model.parameters(), lr=0.0003)

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=100, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

model.to(device)

train(model, optimizer, torch.nn.CrossEntropyLoss(),train_loader, valid_loader, epochs=500, device=device)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to C:\Users\sebas/.cache\torch\hub\v0.10.0.zip


Epoch: 1, Training Loss: 6.75, Validation Loss: 6.52, accuracy = 0.00
Epoch: 2, Training Loss: 6.25, Validation Loss: 5.93, accuracy = 0.09
Epoch: 3, Training Loss: 5.78, Validation Loss: 5.22, accuracy = 0.39
Epoch: 4, Training Loss: 5.31, Validation Loss: 4.92, accuracy = 0.46
Epoch: 5, Training Loss: 4.83, Validation Loss: 4.19, accuracy = 0.61
Epoch: 6, Training Loss: 4.38, Validation Loss: 3.55, accuracy = 0.65
Epoch: 7, Training Loss: 3.92, Validation Loss: 3.41, accuracy = 0.54
Epoch: 8, Training Loss: 3.47, Validation Loss: 3.19, accuracy = 0.48
Epoch: 9, Training Loss: 3.05, Validation Loss: 2.89, accuracy = 0.43
Epoch: 10, Training Loss: 2.74, Validation Loss: 2.35, accuracy = 0.54
Epoch: 11, Training Loss: 2.42, Validation Loss: 2.24, accuracy = 0.46
Epoch: 12, Training Loss: 2.15, Validation Loss: 1.80, accuracy = 0.48
Epoch: 13, Training Loss: 1.93, Validation Loss: 1.79, accuracy = 0.57
Epoch: 14, Training Loss: 1.77, Validation Loss: 1.69, accuracy = 0.50
Epoch: 15, Trai

KeyboardInterrupt: 

In [ ]:
# Function to test the model
def test(model, test_loader, loss_fn, device="cpu"):
    model.eval()
    test_loss = 0.0
    num_correct = 0
    num_examples = 0

    for batch in test_loader:
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        output = model(inputs)
        loss = loss_fn(output, targets)
        test_loss += loss.item() * inputs.size(0)
        correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
        num_correct += torch.sum(correct).item()
        num_examples += correct.shape[0]

    test_loss /= len(test_loader.dataset)
    accuracy = num_correct / num_examples
    print('Test Loss: {:.2f}, Test Accuracy: {:.2f}'.format(test_loss, accuracy))

test(model, test_loader, torch.nn.CrossEntropyLoss(), device=device)


Test Loss: 0.77, Test Accuracy: 0.57
